# 07 Semantic Kernel | 07 RAG Pattern

## Azure Environment

To execute the sample code Azure service specific information like endpoint, api key etc. is needed ([Details and instructions can be found here](../01_DemoEnvironment/01_Environment.ipynb)) 

## Step 1: Create Semantic Kernel instance

An instance of Semantic Kernel can be created using the KernelBuilder object provided by the Semantic Kernel SDK. It acts as the centralized point for all .NET functionality that want to interact with Semantic Kernel functionality or concepts.

It abstracts e.g., models from the OpenAI GPT family and can communicate with Azure OpenAI LLM instances as well as LLMs deployed on OpenAI. In the sample we use models deployed on Azure.

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.3.0"
#r "nuget: Microsoft.SemanticKernel.Core, 1.3.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.Memory, 1.3.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Connectors.Sqlite, 1.3.0-alpha"
#r "nuget: DotNetEnv, 2.5.0"

using System;
using System.IO;
using System.Text;
using System.Text.RegularExpressions;
using System.Text.Json;
using System.Text.Json.Serialization;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Connectors.Sqlite;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Plugins.Memory;

using DotNetEnv;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

static string _configurationFile = @"../01_DemoEnvironment/conf/application.env";
Env.Load(_configurationFile);

string oAiApiKey = Environment.GetEnvironmentVariable("SKIT_AOAI_APIKEY") ?? "SKIT_AOAI_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("SKIT_AOAI_ENDPOINT") ?? "SKIT_AOAI_ENDPOINT not found";
string chatCompletionDeploymentName = Environment.GetEnvironmentVariable("SKIT_CHATCOMPLETION_DEPLOYMENTNAME") ?? "SKIT_CHATCOMPLETION_DEPLOYMENTNAME not found";
string adaDeploymentName = Environment.GetEnvironmentVariable("SKIT_EMBEDDING_DEPLOYMENTNAME") ?? "SKIT_EMBEDDING_DEPLOYMENTNAME not found";

#pragma warning disable CS8618,IDE0009,CA1051,CA1050,CA1707,CA2007,VSTHRD111,CS1591,RCS1110,CA5394,SKEXP0001,SKEXP0002,SKEXP0003,SKEXP0004,SKEXP0010,SKEXP0011,SKEXP0012,SKEXP0020,SKEXP0021,SKEXP0022,SKEXP0023,SKEXP0024,SKEXP0025,SKEXP0026,SKEXP0027,SKEXP0028,SKEXP0029,SKEXP0030,SKEXP0031,SKEXP0032,SKEXP0040,SKEXP0041,SKEXP0042,SKEXP0050,SKEXP0051,SKEXP0052,SKEXP0053,SKEXP0054,SKEXP0055,SKEXP0060,SKEXP0061,SKEXP0101,SKEXP0102
var kernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(chatCompletionDeploymentName, oAiEndpoint, oAiApiKey)
    .AddAzureOpenAITextEmbeddingGeneration(adaDeploymentName, oAiEndpoint, oAiApiKey)
    .Build();


Expected output
```
Installed Packages
dotenv.net, 2.5.0
Microsoft.SemanticKernel, 1.3.0
Microsoft.SemanticKernel.Connectors.Sqlite, 1.3.0-alpha
Microsoft.SemanticKernel.Core, 1.3.0
Microsoft.SemanticKernel.Plugins.Memory, 1.3.0-alpha
```

## Step 2: Get an SK memory service to handle embeddings

In [ ]:
#pragma warning disable CS8618,IDE0009,CA1051,CA1050,CA1707,CA2007,VSTHRD111,CS1591,RCS1110,CA5394,SKEXP0001,SKEXP0002,SKEXP0003,SKEXP0004,SKEXP0010,SKEXP0011,SKEXP0012,SKEXP0020,SKEXP0021,SKEXP0022,SKEXP0023,SKEXP0024,SKEXP0025,SKEXP0026,SKEXP0027,SKEXP0028,SKEXP0029,SKEXP0030,SKEXP0031,SKEXP0032,SKEXP0040,SKEXP0041,SKEXP0042,SKEXP0050,SKEXP0051,SKEXP0052,SKEXP0053,SKEXP0054,SKEXP0055,SKEXP0060,SKEXP0061,SKEXP0101,SKEXP0102
VolatileMemoryStore memoryStore = new VolatileMemoryStore();

#pragma warning disable CS8618,IDE0009,CA1051,CA1050,CA1707,CA2007,VSTHRD111,CS1591,RCS1110,CA5394,SKEXP0001,SKEXP0002,SKEXP0003,SKEXP0004,SKEXP0010,SKEXP0011,SKEXP0012,SKEXP0020,SKEXP0021,SKEXP0022,SKEXP0023,SKEXP0024,SKEXP0025,SKEXP0026,SKEXP0027,SKEXP0028,SKEXP0029,SKEXP0030,SKEXP0031,SKEXP0032,SKEXP0040,SKEXP0041,SKEXP0042,SKEXP0050,SKEXP0051,SKEXP0052,SKEXP0053,SKEXP0054,SKEXP0055,SKEXP0060,SKEXP0061,SKEXP0101,SKEXP0102
AzureOpenAITextEmbeddingGenerationService embeddingGenerator = new AzureOpenAITextEmbeddingGenerationService(
    adaDeploymentName, 
    oAiEndpoint, 
    oAiApiKey
);

// The combination of the text embedding generator and the memory store makes up the 'SemanticTextMemory' object used to store and retrieve memories.
SemanticTextMemory semanticTextMemory = new(memoryStore, embeddingGenerator);

Console.WriteLine($"Memory Store created...");
Console.WriteLine($"Semantic Text Memory created...");


Expected output

```
Memory Store created...
Semantic Text Memory created...
```

## Step 3: Ingestion

### Step 3.1: Read the data

In [ ]:
string assetsFolder = @"../../assets";
string groundingDataFile = Path.Combine(assetsFolder, "docs", "06_SemanticKernel", "groundingData.txt");

string groundingData = File.ReadAllText(groundingDataFile);

Console.WriteLine($"Grounding Data loaded...");

Expected output
```
Grounding data loaded...
```

### Step 3.2: Break the text into chunks by paragraph

In [ ]:
// Define Chunk helper object
public record Chunk(
    [property: JsonPropertyName("id")] string Id,
    [property: JsonPropertyName("text")] string Text);

public class ChunkInfo
{
    public int Index { get; set; }
    public string? Content { get; set; }
    public string? Summary { get; set; }
    public int Words { get; set; }
    public int Characters { get; set; }
    public int Tokens { get; set; }
}

// Keep a list of chunks
List<Chunk> chunks = new List<Chunk>();

// For each learning process the chunks
int chunkId = 1000;
foreach(var chunk in groundingData.Split("\n\n"))
{    
    // Add the chunk to the list
    chunks.Add(new Chunk("RAG-DATA-"+chunkId.ToString(), chunk));
    chunkId++;
}

Console.WriteLine($"Grounding chunks created...");

Expected output
```
Grounding data chunks created...
```

### Step 3.2: Embed and save the text chunk and embedding as a SK Memory

In [ ]:
string memoryCollectionName = "DemoCollection";

// Create an embedding generator to use for semantic memory.
foreach(Chunk chunk in chunks)
{    
    await semanticTextMemory.SaveInformationAsync(memoryCollectionName, id: chunk.Id, text: chunk.Text);
}

Console.WriteLine($"Grounding chunks saved to textMemory instance...");

Expected output
```
Grounding chunks saved to textMemory instance...
```

## Step 4: Grounding

### Step 4.1 Retrieve the memory based on a user's question, a relevance threshold and a limit

In [ ]:
//var query = await InteractiveKernel.GetInputAsync("What is your query?");
string question = "What is the chemical composition of water?";

#pragma warning disable CS8618,IDE0009,CA1051,CA1050,CA1707,CA2007,VSTHRD111,CS1591,RCS1110,CA5394,SKEXP0001,SKEXP0002,SKEXP0003,SKEXP0004,SKEXP0010,SKEXP0011,SKEXP0012,SKEXP0020,SKEXP0021,SKEXP0022,SKEXP0023,SKEXP0024,SKEXP0025,SKEXP0026,SKEXP0027,SKEXP0028,SKEXP0029,SKEXP0030,SKEXP0031,SKEXP0032,SKEXP0040,SKEXP0041,SKEXP0042,SKEXP0050,SKEXP0051,SKEXP0052,SKEXP0053,SKEXP0054,SKEXP0055,SKEXP0060,SKEXP0061,SKEXP0101,SKEXP0102
IAsyncEnumerable<MemoryQueryResult> queryResults =
                semanticTextMemory.SearchAsync(memoryCollectionName, question, limit: 3, minRelevanceScore: 0.77);

Console.WriteLine($"semanticTextMemory queried for grounding information:");
await foreach (var result in queryResults)
{
    Console.WriteLine($"Relevance: {result.Relevance}; Text: {result.Metadata.Text}");
}


Expected output
```
semanticTextMemory queried for grounding information:
Relevance: 0.8762762546539307; Text: Water is an inorganic...
Relevance: 0.8270527720451355; Text: Water covers about 71% of ...
Relevance: 0.8201247453689575; Text: Because Earth's environment is ...
```

### Step 4.2: Collect the text in the memories to augment the prompt

In [ ]:
// Keep the text for the recalled memories
StringBuilder memoryText = new StringBuilder();

#pragma warning disable CS8618,IDE0009,CA1051,CA1050,CA1707,CA2007,VSTHRD111,CS1591,RCS1110,CA5394,SKEXP0001,SKEXP0002,SKEXP0003,SKEXP0004,SKEXP0010,SKEXP0011,SKEXP0012,SKEXP0020,SKEXP0021,SKEXP0022,SKEXP0023,SKEXP0024,SKEXP0025,SKEXP0026,SKEXP0027,SKEXP0028,SKEXP0029,SKEXP0030,SKEXP0031,SKEXP0032,SKEXP0040,SKEXP0041,SKEXP0042,SKEXP0050,SKEXP0051,SKEXP0052,SKEXP0053,SKEXP0054,SKEXP0055,SKEXP0060,SKEXP0061,SKEXP0101,SKEXP0102
await foreach (MemoryQueryResult memoryQueryResult in queryResults)
{
    // Append the text
    memoryText.Append(memoryQueryResult.Metadata.Text+"\n\n");
}

// Final augmented text
var promptContext = memoryText.ToString();

Console.WriteLine($"Prompt context created...");

Expected output: 
```
Prompt context created...
```



## Step 5: Process the Prompt and Completion

- At this point the Prompt includes the original user question plus the text from the embeddings

In [ ]:
const string promptTemplate = "{{$input}}\n\nText:\n\"\"\"{{$context}}\n\"\"\"Use only the provided text.";
var semanticFunction = kernel.CreateFunctionFromPrompt(promptTemplate, new OpenAIPromptExecutionSettings() { MaxTokens = 100, Temperature = 0.3 });
var arguments = new KernelArguments()
        {
            ["input"] = question,
            ["context"] = promptContext
        };

var result = await kernel.InvokeAsync(semanticFunction, arguments);

Console.WriteLine(result);        

Expected output:
```
The chemical composition of water is H2O ...
```

